<a href="https://colab.research.google.com/github/yilmajung/belief_and_llms_v0/blob/main/5_1_contrastive_demographic_space.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Phase 5.1: Contrastive Demographic Space

Phase 5 PCA on 120 "X vs. generic" demographic vectors was dominated by a shared "demographic-ness"
direction (mean cosine sim ~0.39), producing a PC1 that captures attitudinal specificity rather than
political content, and near-zero GSS validation (r~0.04).

This notebook applies two approaches to strip the shared baseline and reveal cleaner demographic axes:

**Approach 2 — Within-Category Centering:**
For each of the 25 categories, subtract the category mean vector. This removes the shared
"demographic-ness" direction within each category, leaving only within-category contrasts.

**Approach 3 — Explicit Contrastive Pairs:**
Compute all C(n,2) pairwise differences within each category: `v_A - v_B`. This directly
produces oppositional vectors that encode "A vs B" rather than "A vs generic."

**No GPU required** — PCA on small matrices is CPU-only.

In [ ]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_similarity
from matplotlib.cm import get_cmap
from itertools import combinations
from scipy import stats
import os

In [ ]:
# Mount Google Drive (Colab) or use local path
try:
    from google.colab import drive
    drive.mount("/content/drive")
    BASE_DIR = "/content/drive/MyDrive/belief_and_llms_v0"
except ImportError:
    BASE_DIR = "."

VECTOR_DIR = os.path.join(BASE_DIR, "vectors")
DATA_DIR = os.path.join(BASE_DIR, "data")

# Reference layer: 13 (peak delta magnitude from Phase 3 analysis)
REF_LAYER = 13

print(f"Base directory: {BASE_DIR}")
print(f"Vector directory: {VECTOR_DIR}")
print(f"Reference layer: {REF_LAYER}")

# Load vectors for the reference layer
vectors = torch.load(
    os.path.join(VECTOR_DIR, f"gss_demographic_vectors_layer{REF_LAYER}.pt"),
    map_location="cpu",
    weights_only=True,
)

# Build sorted labels, categories, and unnormalized matrix
labels = sorted(vectors.keys())
categories = [label.split('_')[0] for label in labels]
unique_cats = sorted(set(categories))

# Reconstruct raw (unnormalized) vectors: direction * magnitude
X = np.array([
    (vectors[lab]['vector'] * vectors[lab]['magnitude']).numpy()
    for lab in labels
])
magnitudes = np.array([vectors[lab]['magnitude'] for lab in labels])

print(f"\nLoaded {len(vectors)} demographic vectors from layer {REF_LAYER}")
print(f"Vector dimension: {X.shape[1]}")
print(f"Matrix shape: {X.shape}")
print(f"Categories ({len(unique_cats)}): {unique_cats}")

# Color palette for 25 categories (tab20 + tab20b)
n_cats = len(unique_cats)
tab20 = [get_cmap('tab20')(i / 20) for i in range(20)]
tab20b = [get_cmap('tab20b')(i / 20) for i in range(20)]
all_colors = tab20 + tab20b[:n_cats - 20] if n_cats > 20 else tab20[:n_cats]
cat_to_color = {cat: all_colors[i] for i, cat in enumerate(unique_cats)}

## Approach 2: Within-Category Centering

For each of the 25 categories, compute the mean vector of its members and subtract it.
This removes the shared "X vs generic" direction within each category, isolating
within-category contrasts.

In [ ]:
# Within-category centering
X_centered = np.copy(X)
cat_means = {}

for cat in unique_cats:
    mask = [c == cat for c in categories]
    indices = [i for i, m in enumerate(mask) if m]
    cat_mean = X[indices].mean(axis=0)
    cat_means[cat] = cat_mean
    X_centered[indices] -= cat_mean

# Verify centering: per-category means should be zero vectors
print("Verification: per-category mean norms after centering")
print("=" * 50)
for cat in unique_cats:
    mask = [c == cat for c in categories]
    indices = [i for i, m in enumerate(mask) if m]
    mean_norm = np.linalg.norm(X_centered[indices].mean(axis=0))
    n_members = len(indices)
    print(f"  {cat:15s}: mean_norm = {mean_norm:.2e}, n_members = {n_members}")

print(f"\nCentered matrix shape: {X_centered.shape}")
print(f"Original matrix norm: {np.linalg.norm(X, 'fro'):.2f}")
print(f"Centered matrix norm: {np.linalg.norm(X_centered, 'fro'):.2f}")

In [ ]:
# PCA on centered matrix
n_components = min(30, len(labels))

# Original PCA (for comparison)
pca_orig = PCA(n_components=n_components)
X_pca_orig = pca_orig.fit_transform(X)

# Centered PCA
pca_centered = PCA(n_components=n_components)
X_pca_centered = pca_centered.fit_transform(X_centered)

# Scree plot comparison: original vs centered
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Scree plot (centered)
axes[0].bar(range(1, 21), pca_centered.explained_variance_ratio_[:20], color='steelblue', alpha=0.8)
axes[0].set_xlabel('Principal Component', fontsize=12)
axes[0].set_ylabel('Explained Variance Ratio', fontsize=12)
axes[0].set_title('Scree Plot (centered vectors)', fontsize=13)
axes[0].set_xticks(range(1, 21))

# Cumulative (centered)
cum_var_c = np.cumsum(pca_centered.explained_variance_ratio_[:20])
axes[1].plot(range(1, 21), cum_var_c, 'bo-', linewidth=2, markersize=6)
axes[1].axhline(0.8, color='red', linestyle='--', alpha=0.5, label='80%')
axes[1].axhline(0.9, color='orange', linestyle='--', alpha=0.5, label='90%')
axes[1].set_xlabel('Number of Components', fontsize=12)
axes[1].set_ylabel('Cumulative Explained Variance', fontsize=12)
axes[1].set_title('Cumulative Variance (centered)', fontsize=13)
axes[1].legend()
axes[1].set_xticks(range(1, 21))

# Comparison: original vs centered
axes[2].plot(range(1, 21), pca_orig.explained_variance_ratio_[:20], 'rs-', label='Original', linewidth=2)
axes[2].plot(range(1, 21), pca_centered.explained_variance_ratio_[:20], 'bo-', label='Centered', linewidth=2)
axes[2].set_xlabel('Principal Component', fontsize=12)
axes[2].set_ylabel('Explained Variance Ratio', fontsize=12)
axes[2].set_title('Original vs Centered', fontsize=13)
axes[2].legend()
axes[2].set_xticks(range(1, 21))

plt.tight_layout()
plt.show()

print(f"\nTop 10 PCs explained variance comparison:")
print(f"{'PC':>4s}  {'Original':>10s}  {'Centered':>10s}  {'Diff':>10s}")
print("-" * 40)
for i in range(10):
    orig_v = pca_orig.explained_variance_ratio_[i]
    cent_v = pca_centered.explained_variance_ratio_[i]
    print(f"PC{i+1:>2d}  {orig_v:10.4f}  {cent_v:10.4f}  {cent_v - orig_v:+10.4f}")

In [ ]:
# Centered PCA: PC1 vs PC2 scatter
df_centered = pd.DataFrame({
    'PC1': X_pca_centered[:, 0],
    'PC2': X_pca_centered[:, 1],
    'PC3': X_pca_centered[:, 2],
    'label': labels,
    'category': categories,
    'short_label': [label.split('_', 1)[1][:30] for label in labels],
})

fig, ax = plt.subplots(figsize=(18, 14))

for cat in unique_cats:
    mask = df_centered['category'] == cat
    ax.scatter(
        df_centered.loc[mask, 'PC1'], df_centered.loc[mask, 'PC2'],
        label=cat, color=cat_to_color[cat], s=80, alpha=0.8,
        edgecolors='white', linewidth=0.5,
    )

for _, row in df_centered.iterrows():
    ax.annotate(
        row['short_label'], (row['PC1'], row['PC2']),
        fontsize=5.5, alpha=0.8,
        xytext=(3, 3), textcoords='offset points',
    )

ax.axhline(0, color='black', linestyle='--', alpha=0.2)
ax.axvline(0, color='black', linestyle='--', alpha=0.2)
ax.set_xlabel(f'PC1 ({pca_centered.explained_variance_ratio_[0]:.1%} variance)', fontsize=14)
ax.set_ylabel(f'PC2 ({pca_centered.explained_variance_ratio_[1]:.1%} variance)', fontsize=14)
ax.set_title(f'Centered Demographic Space: PC1 vs PC2 (Layer {REF_LAYER})', fontsize=16)
ax.legend(bbox_to_anchor=(1.02, 1), loc='upper left', fontsize=8, ncol=2)

plt.tight_layout()
plt.show()

In [ ]:
# Centered PCA: PC Poles (top/bottom 10 on PC1-PC5)
print("CENTERED PCA — PC Poles")
for pc_idx in range(5):
    pc_name = f'PC{pc_idx + 1}'
    projections = X_pca_centered[:, pc_idx]
    sorted_idx = np.argsort(projections)

    print(f"\n{'=' * 70}")
    print(f"{pc_name} — Explained Variance: {pca_centered.explained_variance_ratio_[pc_idx]:.2%}")
    print(f"{'=' * 70}")

    print(f"\n  POSITIVE pole (top 10):")
    for i in sorted_idx[-10:][::-1]:
        print(f"    {projections[i]:+8.4f}  [{categories[i]:12s}]  {labels[i]}")

    print(f"\n  NEGATIVE pole (bottom 10):")
    for i in sorted_idx[:10]:
        print(f"    {projections[i]:+8.4f}  [{categories[i]:12s}]  {labels[i]}")

## Approach 3: Explicit Contrastive Pairs

For each category, compute all C(n,2) pairwise differences: `v_A - v_B`.
These contrastive vectors directly encode "A vs B" within each category,
completely removing any shared baseline.

In [ ]:
# Build contrastive pair matrix
contrastive_vectors = []
contrastive_labels = []
contrastive_categories = []

label_to_idx = {lab: i for i, lab in enumerate(labels)}

for cat in unique_cats:
    # Get sorted members of this category
    members = sorted([lab for lab in labels if lab.split('_')[0] == cat])
    
    # All ordered pairs (combinations)
    for a, b in combinations(members, 2):
        idx_a = label_to_idx[a]
        idx_b = label_to_idx[b]
        v_contrast = X[idx_a] - X[idx_b]
        
        short_a = a.split('_', 1)[1][:20]
        short_b = b.split('_', 1)[1][:20]
        pair_label = f"{cat}: {short_a} vs {short_b}"
        
        contrastive_vectors.append(v_contrast)
        contrastive_labels.append(pair_label)
        contrastive_categories.append(cat)

X_contrastive = np.array(contrastive_vectors)

print(f"Contrastive matrix shape: {X_contrastive.shape}")
print(f"Total contrastive pairs: {len(contrastive_labels)}")
print(f"\nPairs per category:")
for cat in unique_cats:
    n_pairs = sum(1 for c in contrastive_categories if c == cat)
    n_members = sum(1 for c in categories if c == cat)
    print(f"  {cat:15s}: {n_members} members -> C({n_members},2) = {n_pairs} pairs")

In [ ]:
# PCA on contrastive matrix
n_comp_contrast = min(30, len(contrastive_labels))
pca_contrast = PCA(n_components=n_comp_contrast)
X_pca_contrast = pca_contrast.fit_transform(X_contrastive)

# Scree plot
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].bar(range(1, 21), pca_contrast.explained_variance_ratio_[:20], color='steelblue', alpha=0.8)
axes[0].set_xlabel('Principal Component', fontsize=12)
axes[0].set_ylabel('Explained Variance Ratio', fontsize=12)
axes[0].set_title('Scree Plot (contrastive pairs)', fontsize=13)
axes[0].set_xticks(range(1, 21))

cum_var_k = np.cumsum(pca_contrast.explained_variance_ratio_[:20])
axes[1].plot(range(1, 21), cum_var_k, 'bo-', linewidth=2, markersize=6)
axes[1].axhline(0.8, color='red', linestyle='--', alpha=0.5, label='80%')
axes[1].axhline(0.9, color='orange', linestyle='--', alpha=0.5, label='90%')
axes[1].set_xlabel('Number of Components', fontsize=12)
axes[1].set_ylabel('Cumulative Explained Variance', fontsize=12)
axes[1].set_title('Cumulative Variance (contrastive)', fontsize=13)
axes[1].legend()
axes[1].set_xticks(range(1, 21))

plt.tight_layout()
plt.show()

print(f"\nTop 10 PCs explained variance (contrastive):")
for i in range(10):
    print(f"  PC{i+1}: {pca_contrast.explained_variance_ratio_[i]:.4f} "
          f"(cumulative: {sum(pca_contrast.explained_variance_ratio_[:i+1]):.4f})")

In [ ]:
# Contrastive PCA: PC1 vs PC2 scatter
df_contrast = pd.DataFrame({
    'PC1': X_pca_contrast[:, 0],
    'PC2': X_pca_contrast[:, 1],
    'label': contrastive_labels,
    'category': contrastive_categories,
})

fig, ax = plt.subplots(figsize=(18, 14))

for cat in unique_cats:
    mask = df_contrast['category'] == cat
    if mask.any():
        ax.scatter(
            df_contrast.loc[mask, 'PC1'], df_contrast.loc[mask, 'PC2'],
            label=cat, color=cat_to_color[cat], s=60, alpha=0.7,
            edgecolors='white', linewidth=0.5,
        )

# Annotate politically relevant pairs
political_keywords = [
    'Democrat', 'Republican', 'Liberal', 'Conservative',
    'gun owner', 'believes in God', 'does not believe',
    'rural', 'central city',
]
for _, row in df_contrast.iterrows():
    if any(kw in row['label'] for kw in political_keywords):
        ax.annotate(
            row['label'], (row['PC1'], row['PC2']),
            fontsize=4.5, alpha=0.7,
            xytext=(3, 3), textcoords='offset points',
        )

ax.axhline(0, color='black', linestyle='--', alpha=0.2)
ax.axvline(0, color='black', linestyle='--', alpha=0.2)
ax.set_xlabel(f'PC1 ({pca_contrast.explained_variance_ratio_[0]:.1%} variance)', fontsize=14)
ax.set_ylabel(f'PC2 ({pca_contrast.explained_variance_ratio_[1]:.1%} variance)', fontsize=14)
ax.set_title(f'Contrastive Demographic Space: PC1 vs PC2 (Layer {REF_LAYER})', fontsize=16)
ax.legend(bbox_to_anchor=(1.02, 1), loc='upper left', fontsize=8, ncol=2)

plt.tight_layout()
plt.show()

In [ ]:
# Contrastive PCA: PC Poles (top/bottom 10 on PC1-PC5)
print("CONTRASTIVE PCA — PC Poles")
for pc_idx in range(5):
    pc_name = f'PC{pc_idx + 1}'
    projections = X_pca_contrast[:, pc_idx]
    sorted_idx = np.argsort(projections)

    print(f"\n{'=' * 80}")
    print(f"{pc_name} — Explained Variance: {pca_contrast.explained_variance_ratio_[pc_idx]:.2%}")
    print(f"{'=' * 80}")

    print(f"\n  POSITIVE pole (top 10):")
    for i in sorted_idx[-10:][::-1]:
        print(f"    {projections[i]:+8.4f}  [{contrastive_categories[i]:12s}]  {contrastive_labels[i]}")

    print(f"\n  NEGATIVE pole (bottom 10):")
    for i in sorted_idx[:10]:
        print(f"    {projections[i]:+8.4f}  [{contrastive_categories[i]:12s}]  {contrastive_labels[i]}")

## Side-by-Side Comparison: Original vs Centered vs Contrastive

In [ ]:
# Three-panel figure: Original, Centered, Contrastive — all PC1 vs PC2
fig, axes = plt.subplots(1, 3, figsize=(24, 8))

# Panel 1: Original PCA
ax = axes[0]
for cat in unique_cats:
    mask = [c == cat for c in categories]
    indices = [i for i, m in enumerate(mask) if m]
    ax.scatter(
        X_pca_orig[indices, 0], X_pca_orig[indices, 1],
        label=cat, color=cat_to_color[cat], s=50, alpha=0.7,
        edgecolors='white', linewidth=0.3,
    )
ax.axhline(0, color='black', linestyle='--', alpha=0.2)
ax.axvline(0, color='black', linestyle='--', alpha=0.2)
ax.set_xlabel(f"PC1 ({pca_orig.explained_variance_ratio_[0]:.1%})")
ax.set_ylabel(f"PC2 ({pca_orig.explained_variance_ratio_[1]:.1%})")
ax.set_title('Original (X vs generic)', fontsize=14)

# Panel 2: Centered PCA
ax = axes[1]
for cat in unique_cats:
    mask = [c == cat for c in categories]
    indices = [i for i, m in enumerate(mask) if m]
    ax.scatter(
        X_pca_centered[indices, 0], X_pca_centered[indices, 1],
        label=cat, color=cat_to_color[cat], s=50, alpha=0.7,
        edgecolors='white', linewidth=0.3,
    )
ax.axhline(0, color='black', linestyle='--', alpha=0.2)
ax.axvline(0, color='black', linestyle='--', alpha=0.2)
ax.set_xlabel(f"PC1 ({pca_centered.explained_variance_ratio_[0]:.1%})")
ax.set_ylabel(f"PC2 ({pca_centered.explained_variance_ratio_[1]:.1%})")
ax.set_title('Within-Category Centered', fontsize=14)

# Panel 3: Contrastive PCA
ax = axes[2]
for cat in unique_cats:
    mask = [c == cat for c in contrastive_categories]
    if any(mask):
        indices = [i for i, m in enumerate(mask) if m]
        ax.scatter(
            X_pca_contrast[indices, 0], X_pca_contrast[indices, 1],
            label=cat, color=cat_to_color[cat], s=40, alpha=0.7,
            edgecolors='white', linewidth=0.3,
        )
ax.axhline(0, color='black', linestyle='--', alpha=0.2)
ax.axvline(0, color='black', linestyle='--', alpha=0.2)
ax.set_xlabel(f"PC1 ({pca_contrast.explained_variance_ratio_[0]:.1%})")
ax.set_ylabel(f"PC2 ({pca_contrast.explained_variance_ratio_[1]:.1%})")
ax.set_title('Explicit Contrastive Pairs', fontsize=14)

axes[-1].legend(bbox_to_anchor=(1.02, 1), loc='upper left', fontsize=6, ncol=2)
plt.suptitle(f'Three PCA Approaches Compared (Layer {REF_LAYER})', fontsize=16)
plt.tight_layout()
plt.show()

# Print comparison table
print(f"\nExplained Variance Comparison:")
print(f"{'PC':>4s}  {'Original':>10s}  {'Centered':>10s}  {'Contrastive':>12s}")
print("-" * 42)
for i in range(10):
    print(f"PC{i+1:>2d}  {pca_orig.explained_variance_ratio_[i]:10.4f}  "
          f"{pca_centered.explained_variance_ratio_[i]:10.4f}  "
          f"{pca_contrast.explained_variance_ratio_[i]:12.4f}")

## Liberal-Conservative Hypothesis Test

For each PCA space, compute mean PC1 for liberal vs conservative clusters
and culture-war demographics. Check which approach best separates them.

In [ ]:
# Define clusters
liberal_labels = [
    'PartyID_Strong Democrat',
    'PolViews_person with a liberal political view',
    'PolViews_person with an extremely liberal political view',
]
conservative_labels = [
    'PartyID_Strong Republican',
    'PolViews_person with a conservative political view',
    'PolViews_person with an extremely conservative political view',
]
culture_war = {
    'Gun owner vs Non-owner': ('OwnGun_gun owner', 'OwnGun_person who does not own a gun'),
    'Firm believer vs Non-believer': ('God_person who firmly believes in God', 'God_person who does not believe in God'),
    'Rural vs Urban': ('Urbanity_person living in a rural area', 'Urbanity_person living in a central city'),
}

def get_mean_pc1(pca_result, target_labels, all_labels):
    """Get mean PC1 projection for a set of labels."""
    vals = []
    for lab in target_labels:
        if lab in all_labels:
            idx = all_labels.index(lab)
            vals.append(pca_result[idx, 0])
    return np.mean(vals) if vals else np.nan

# --- Original PCA ---
print("LIBERAL-CONSERVATIVE SEPARATION ON PC1")
print("=" * 70)

approaches = [
    ('Original', X_pca_orig, labels),
    ('Centered', X_pca_centered, labels),
]

results = []
for name, pca_result, lab_list in approaches:
    lib_mean = get_mean_pc1(pca_result, liberal_labels, lab_list)
    con_mean = get_mean_pc1(pca_result, conservative_labels, lab_list)
    separation = abs(lib_mean - con_mean)
    results.append({'Approach': name, 'Liberal mean PC1': lib_mean,
                    'Conservative mean PC1': con_mean, 'Separation': separation})

    print(f"\n{name}:")
    print(f"  Liberal cluster mean PC1:      {lib_mean:+.4f}")
    print(f"  Conservative cluster mean PC1: {con_mean:+.4f}")
    print(f"  Separation (|diff|):           {separation:.4f}")

    print(f"\n  Culture war demographics on PC1:")
    for desc, (lab_a, lab_b) in culture_war.items():
        if lab_a in lab_list and lab_b in lab_list:
            idx_a = lab_list.index(lab_a)
            idx_b = lab_list.index(lab_b)
            val_a = pca_result[idx_a, 0]
            val_b = pca_result[idx_b, 0]
            print(f"    {desc:35s}: {val_a:+.4f} vs {val_b:+.4f}  (diff={val_a - val_b:+.4f})")

print(f"\n{'=' * 70}")
print("\nSummary Table:")
df_results = pd.DataFrame(results)
print(df_results.to_string(index=False))

## GSS Phi-Coefficient Validation

Compare cosine similarity between centered vectors with GSS phi-coefficients.
If centering removes shared baseline noise, the correlation should improve from the
Phase 5 baseline (r~0.04).

In [ ]:
# GSS-to-vector mapping (same as Phase 5)
GSS_TO_VECTOR = {
    # Race
    'race_Black': 'Race_Black person',
    'race_White': 'Race_White person',
    'race_Other': 'Race_person of different race than Black or White',
    # PartyID
    'partyid_Strong democrat': 'PartyID_Strong Democrat',
    'partyid_Strong republican': 'PartyID_Strong Republican',
    'partyid_Not very strong democrat': 'PartyID_Democrat',
    'partyid_Not very strong republican': 'PartyID_Republican',
    'partyid_Independent, close to democrat': 'PartyID_Independent leaning Democrat',
    'partyid_Independent, close to republican': 'PartyID_Independent leaning Republican',
    'partyid_Independent (neither, no response)': 'PartyID_Political Independent',
    'partyid_Other party': 'PartyID_Other',
    # Sex
    'sex_MALE': 'Sex_man',
    'sex_FEMALE': 'Sex_woman',
    # Degree
    'degree_Less than high school': 'Degree_person with less than high school education',
    'degree_High school': 'Degree_high school graduate',
    'degree_Associate/junior college': 'Degree_person with some college education',
    "degree_Bachelor's": "Degree_person with a bachelor's degree",
    'degree_Graduate': 'Degree_person with a graduate degree',
    # Religion
    'relig_Catholic': 'Religion_Catholic',
    'relig_Protestant': 'Religion_Protestant',
    'relig_Buddhism': 'Religion_Buddhist',
    'relig_Jewish': 'Religion_Jewish person',
    'relig_Muslim/Islam': 'Religion_Muslim',
    'relig_Hinduism': 'Religion_Hindu',
    'relig_Other': 'Religion_person of other religion',
    # PolViews
    'polviews_Extremely liberal': 'PolViews_person with an extremely liberal political view',
    'polviews_Liberal': 'PolViews_person with a liberal political view',
    'polviews_Slightly liberal': 'PolViews_person with a slightly liberal political view',
    'polviews_Moderate, middle of the road': 'PolViews_person with a neutral political view',
    'polviews_Slightly conservative': 'PolViews_person with a slightly conservative political view',
    'polviews_Conservative': 'PolViews_person with a conservative political view',
    'polviews_Extremely conservative': 'PolViews_person with an extremely conservative political view',
    # Generation
    'generation_Generation Z': 'Generation_person from Generation Z',
    'generation_Millennial': 'Generation_Millennial',
    'generation_Generation X': 'Generation_person from Generation X',
    'generation_Baby Boomer': 'Generation_Baby Boomer',
    'generation_Silent Generation': 'Generation_person from the Silent Generation',
    # Marital
    'marital_Married': 'Marital_married person',
    'marital_Divorced': 'Marital_divorced person',
    'marital_Widowed': 'Marital_widowed person',
    'marital_Separated': 'Marital_separated person',
    'marital_Never married': 'Marital_person who has never been married',
    # Age
    'age_group_Youth (15-24)': 'Age_young person aged 15 to 24',
    'age_group_Young Adult (25-39)': 'Age_young adult aged 25 to 39',
    'age_group_Middle Adult (40-64)': 'Age_middle-aged adult aged 40 to 64',
    'age_group_Elderly (65+)': 'Age_elderly person aged 65 or older',
    # Children
    'childs_group_No children': 'Children_person with no children',
    'childs_group_1-2 children': 'Children_person with one or two children',
    'childs_group_3 or more children': 'Children_person with three or more children',
    # Immigration
    'immig_gen_1st generation immigrant': 'ImmigGen_first-generation immigrant to the United States',
    'immig_gen_2nd generation immigrant': 'ImmigGen_second-generation immigrant in the United States',
    'immig_gen_3rd+ generation American': 'ImmigGen_third-generation or later American',
    # Region (grew up)
    'reg16_Northeast': 'Region16_person who grew up in the Northeast',
    'reg16_Midwest': 'Region16_person who grew up in the Midwest',
    'reg16_South': 'Region16_person who grew up in the South',
    'reg16_West': 'Region16_person who grew up in the West',
    'reg16_Foreign': 'Region16_person who grew up in a foreign country',
    # Family income
    'income_group_Income under $10k': 'FamilyIncome_person with a family income under $10,000',
    'income_group_Income $10k-$25k': 'FamilyIncome_person with a family income between $10,000 and $25,000',
    'income_group_Income $25k or more': 'FamilyIncome_person with a family income of $25,000 or more',
    # Region (current)
    'region_Northeast': 'Region_person living in the Northeast',
    'region_Midwest': 'Region_person living in the Midwest',
    'region_South': 'Region_person living in the South',
    'region_West': 'Region_person living in the West',
    # Urbanity
    'srcbelt_group_Central city': 'Urbanity_person living in a central city',
    'srcbelt_group_Suburban': 'Urbanity_person living in the suburbs',
    'srcbelt_group_Other urban': 'Urbanity_person living in a small city or town',
    'srcbelt_group_Rural': 'Urbanity_person living in a rural area',
    # Happiness
    'happy_Very happy': 'Happy_very happy person',
    'happy_Pretty happy': 'Happy_fairly happy person',
    'happy_Not too happy': 'Happy_unhappy person',
    # Health
    'health_Excellent': 'Health_person in excellent health',
    'health_Good': 'Health_person in good health',
    'health_Fair': 'Health_person in fair health',
    'health_Poor': 'Health_person in poor health',
    # Life excitement
    'life_Exciting': 'Life_person who finds life exciting',
    'life_Routine': 'Life_person who finds life routine',
    'life_Dull': 'Life_person who finds life dull',
    # Job satisfaction
    'satjob_group_Very satisfied': 'SatJob_person very satisfied with their job',
    'satjob_group_Moderately satisfied': 'SatJob_person moderately satisfied with their job',
    'satjob_group_Dissatisfied': 'SatJob_person dissatisfied with their job',
    # Social class
    'class__Lower class': 'Class_lower class person',
    'class__Working class': 'Class_working class person',
    'class__Middle class': 'Class_middle class person',
    'class__Upper class': 'Class_upper class person',
    # Financial satisfaction
    'satfin_Pretty well satisfied': 'SatFin_person satisfied with their financial situation',
    'satfin_More or less satisfied': 'SatFin_person somewhat satisfied with their financial situation',
    'satfin_Not satisfied at all': 'SatFin_person not satisfied with their financial situation',
    # Gun ownership
    'owngun_YES': 'OwnGun_gun owner',
    'owngun_NO': 'OwnGun_person who does not own a gun',
    # Belief in God
    'god_No doubts': 'God_person who firmly believes in God',
    'god_Believe with doubts': 'God_person who believes in God but with some doubts',
    'god_Believe sometimes': 'God_person who sometimes believes in God',
    "god_Don't know, no way to find out": 'God_agnostic person',
    'god_Higher power': 'God_person who believes in a higher power but not a personal God',
    "god_Don't believe": 'God_person who does not believe in God',
    # Occupation
    'occ_group_Service': 'Occupation_person working in a service occupation',
    'occ_group_Natural Resources, Construction, and Maintenance': 'Occupation_person working in construction, maintenance, or natural resources',
    'occ_group_Office and Administrative Support': 'Occupation_person working in an office or administrative support role',
    'occ_group_Management, Business, and Financial': 'Occupation_person working in management, business, or finance',
    'occ_group_Education, Legal, Community Service, Arts, and Media': 'Occupation_person working in education, law, community service, arts, or media',
    'occ_group_Computer, Engineering, and Science': 'Occupation_person working in computer science, engineering, or science',
    'occ_group_Military': 'Occupation_person serving in the military',
    'occ_group_Production, Transportation, and Material Moving': 'Occupation_person working in production or transportation',
    'occ_group_Sales and Related': 'Occupation_person working in sales',
    'occ_group_Healthcare Practitioners and Technical': 'Occupation_healthcare practitioner',
    # Industry
    'indus_group_Health Care and Social Assistance': 'Industry_person working in healthcare or social assistance',
    'indus_group_Wholesale Trade': 'Industry_person working in wholesale trade',
    'indus_group_Manufacturing': 'Industry_person working in manufacturing',
    'indus_group_Construction': 'Industry_person working in the construction industry',
    'indus_group_Finance and Insurance, and Real Estate': 'Industry_person working in finance, insurance, or real estate',
    'indus_group_Arts, Entertainment, Recreation, and Accommodation and Food Services': 'Industry_person working in arts, entertainment, or food services',
    'indus_group_Information': 'Industry_person working in the information industry',
    'indus_group_Educational Services': 'Industry_person working in educational services',
    'indus_group_Public Administration': 'Industry_person working in public administration',
    'indus_group_Professional, Scientific, Management, and Administrative Services': 'Industry_person working in professional or scientific services',
    'indus_group_Transportation and Warehousing, and Utilities': 'Industry_person working in transportation or utilities',
    'indus_group_Retail Trade': 'Industry_person working in retail',
    'indus_group_Other Services (Except Public Administration)': 'Industry_person working in other services',
    'indus_group_Agriculture, Forestry, Fishing and Hunting, and Mining': 'Industry_person working in agriculture or mining',
}

print(f"GSS_TO_VECTOR mapping: {len(GSS_TO_VECTOR)} entries")

In [ ]:
# Load GSS phi-coefficient correlations
corr_df = pd.read_csv(os.path.join(DATA_DIR, "gss_correlation_pairs.csv"))
print(f"Loaded {len(corr_df)} correlation pairs")

# Compute cosine similarities for original and centered vectors
cos_sim_orig = cosine_similarity(X)
cos_sim_centered = cosine_similarity(X_centered)

# Build comparison dataframe
comparison_data = []

for _, row in corr_df.iterrows():
    var1, var2, phi = row['Var1'], row['Var2'], row['Correlation']

    vec_label1 = GSS_TO_VECTOR.get(var1)
    vec_label2 = GSS_TO_VECTOR.get(var2)

    if vec_label1 and vec_label2 and vec_label1 in label_to_idx and vec_label2 in label_to_idx:
        idx1 = label_to_idx[vec_label1]
        idx2 = label_to_idx[vec_label2]
        comparison_data.append({
            'gss_var1': var1, 'gss_var2': var2,
            'vec_label1': vec_label1, 'vec_label2': vec_label2,
            'phi_coefficient': phi,
            'cos_sim_original': cos_sim_orig[idx1, idx2],
            'cos_sim_centered': cos_sim_centered[idx1, idx2],
        })

comp_df = pd.DataFrame(comparison_data)
print(f"Matched {len(comp_df)} out of {len(corr_df)} correlation pairs")

# Compute correlations
pearson_orig = comp_df['phi_coefficient'].corr(comp_df['cos_sim_original'])
spearman_orig = comp_df['phi_coefficient'].corr(comp_df['cos_sim_original'], method='spearman')
pearson_centered = comp_df['phi_coefficient'].corr(comp_df['cos_sim_centered'])
spearman_centered = comp_df['phi_coefficient'].corr(comp_df['cos_sim_centered'], method='spearman')

print(f"\nGSS Validation Results:")
print(f"{'Approach':>15s}  {'Pearson r':>10s}  {'Spearman rho':>13s}")
print("-" * 42)
print(f"{'Original':>15s}  {pearson_orig:10.4f}  {spearman_orig:13.4f}")
print(f"{'Centered':>15s}  {pearson_centered:10.4f}  {spearman_centered:13.4f}")

In [ ]:
# Scatter plots: phi-coefficient vs cosine similarity (original vs centered, side by side)
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

for ax, col, title, r_val in [
    (axes[0], 'cos_sim_original', 'Original Vectors', pearson_orig),
    (axes[1], 'cos_sim_centered', 'Centered Vectors', pearson_centered),
]:
    ax.scatter(comp_df['phi_coefficient'], comp_df[col],
              alpha=0.3, s=20, color='steelblue')

    # Fit line
    z = np.polyfit(comp_df['phi_coefficient'], comp_df[col], 1)
    p = np.poly1d(z)
    x_fit = np.linspace(comp_df['phi_coefficient'].min(), comp_df['phi_coefficient'].max(), 100)
    ax.plot(x_fit, p(x_fit), 'r-', linewidth=2, alpha=0.7)

    ax.set_xlabel('GSS Phi-Coefficient', fontsize=12)
    ax.set_ylabel('Vector Cosine Similarity', fontsize=12)
    ax.set_title(f'{title} (r={r_val:.3f})', fontsize=13)
    ax.axhline(0, color='gray', linestyle='--', alpha=0.3)
    ax.axvline(0, color='gray', linestyle='--', alpha=0.3)

plt.suptitle('GSS Phi-Coefficient vs Vector Cosine Similarity', fontsize=15)
plt.tight_layout()
plt.show()

# Show cross-category vs within-category breakdown
comp_df['same_category'] = [
    row['vec_label1'].split('_')[0] == row['vec_label2'].split('_')[0]
    for _, row in comp_df.iterrows()
]

print(f"\nBreakdown by pair type:")
print(f"{'Type':>20s}  {'N':>5s}  {'Pearson (orig)':>15s}  {'Pearson (cent)':>15s}")
print("-" * 60)
for same_cat, label in [(True, 'Within-category'), (False, 'Cross-category')]:
    sub = comp_df[comp_df['same_category'] == same_cat]
    if len(sub) > 2:
        r_o = sub['phi_coefficient'].corr(sub['cos_sim_original'])
        r_c = sub['phi_coefficient'].corr(sub['cos_sim_centered'])
        print(f"{label:>20s}  {len(sub):5d}  {r_o:15.4f}  {r_c:15.4f}")

## Multi-Layer Comparison (Centered PCA)

Run within-category centering + PCA across layers 5, 9, 13, 17, 20 to check
whether centering improves cross-layer consistency.

In [ ]:
comparison_layers = [5, 9, 13, 17, 20]
layer_results_orig = {}
layer_results_centered = {}

for layer in comparison_layers:
    vecs = torch.load(
        os.path.join(VECTOR_DIR, f"gss_demographic_vectors_layer{layer}.pt"),
        map_location="cpu",
        weights_only=True,
    )

    layer_labels = sorted(vecs.keys())
    layer_cats = [lab.split('_')[0] for lab in layer_labels]
    layer_X = np.array([
        (vecs[lab]['vector'] * vecs[lab]['magnitude']).numpy()
        for lab in layer_labels
    ])

    # Original PCA
    pca_o = PCA(n_components=min(20, len(layer_labels)))
    X_pca_o = pca_o.fit_transform(layer_X)
    layer_results_orig[layer] = {
        'labels': layer_labels, 'categories': layer_cats,
        'X_pca': X_pca_o, 'pca': pca_o,
        'explained_variance': pca_o.explained_variance_ratio_,
    }

    # Within-category centering
    layer_X_c = np.copy(layer_X)
    layer_unique_cats = sorted(set(layer_cats))
    for cat in layer_unique_cats:
        mask = [c == cat for c in layer_cats]
        indices = [i for i, m in enumerate(mask) if m]
        cat_mean = layer_X[indices].mean(axis=0)
        layer_X_c[indices] -= cat_mean

    pca_c = PCA(n_components=min(20, len(layer_labels)))
    X_pca_c = pca_c.fit_transform(layer_X_c)
    layer_results_centered[layer] = {
        'labels': layer_labels, 'categories': layer_cats,
        'X_pca': X_pca_c, 'pca': pca_c,
        'explained_variance': pca_c.explained_variance_ratio_,
    }

    print(f"Layer {layer:2d}:  Original PC1={pca_o.explained_variance_ratio_[0]:.3f}  "
          f"Centered PC1={pca_c.explained_variance_ratio_[0]:.3f}  "
          f"Original top-3={sum(pca_o.explained_variance_ratio_[:3]):.3f}  "
          f"Centered top-3={sum(pca_c.explained_variance_ratio_[:3]):.3f}")

In [ ]:
# Plot: explained variance comparison across layers (original vs centered)
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# PC1-PC10: original (dashed) vs centered (solid)
ax = axes[0]
colors_layer = plt.cm.viridis(np.linspace(0, 1, len(comparison_layers)))
for i, layer in enumerate(comparison_layers):
    ev_o = layer_results_orig[layer]['explained_variance'][:10]
    ev_c = layer_results_centered[layer]['explained_variance'][:10]
    ax.plot(range(1, 11), ev_o, 'o--', color=colors_layer[i], alpha=0.5, linewidth=1, markersize=4)
    ax.plot(range(1, 11), ev_c, 's-', color=colors_layer[i], label=f'Layer {layer}', linewidth=2, markersize=5)
ax.set_xlabel('Principal Component', fontsize=12)
ax.set_ylabel('Explained Variance Ratio', fontsize=12)
ax.set_title('Centered (solid) vs Original (dashed)', fontsize=13)
ax.legend()
ax.set_xticks(range(1, 11))

# Cumulative centered
ax = axes[1]
for i, layer in enumerate(comparison_layers):
    ev_c = layer_results_centered[layer]['explained_variance'][:10]
    ax.plot(range(1, 11), np.cumsum(ev_c), 'o-', color=colors_layer[i],
            label=f'Layer {layer}', linewidth=2, markersize=6)
ax.set_xlabel('Number of Components', fontsize=12)
ax.set_ylabel('Cumulative Explained Variance', fontsize=12)
ax.set_title('Cumulative Variance (Centered PCA)', fontsize=13)
ax.legend()
ax.set_xticks(range(1, 11))

plt.tight_layout()
plt.show()

# Compare PC1 poles across layers (centered)
print("\nCentered PC1 Poles Across Layers")
print("=" * 80)

for layer in comparison_layers:
    r = layer_results_centered[layer]
    projections = r['X_pca'][:, 0]
    sorted_idx = np.argsort(projections)

    top5 = [r['labels'][i].split('_', 1)[1][:30] for i in sorted_idx[-5:][::-1]]
    bot5 = [r['labels'][i].split('_', 1)[1][:30] for i in sorted_idx[:5]]

    print(f"\nLayer {layer} (PC1 = {r['explained_variance'][0]:.1%}):")
    print(f"  + pole: {', '.join(top5)}")
    print(f"  - pole: {', '.join(bot5)}")

## Summary & Interpretation

### Motivation
Phase 5 PCA on the original "X vs. generic" vectors was dominated by a shared baseline
(mean cosine similarity ~0.39 across all pairs). PC1 (23.6%) captured "attitudinal specificity"
rather than the expected liberal-conservative axis, and GSS phi-coefficient validation showed
near-zero correlation (r~0.04).

### Approach 2: Within-Category Centering
- Subtracts the category mean from each vector, removing within-category shared variance
- Preserves the 120-vector structure, so direct comparison with original PCA is possible
- GSS validation uses cosine similarity on centered vectors

### Approach 3: Explicit Contrastive Pairs
- Computes all C(n,2) pairwise differences within each category (~317 vectors)
- Each contrastive vector directly encodes "A vs B" with no shared baseline
- Produces a larger matrix but with purely oppositional information

### Key Questions
1. Does PC1 now separate liberal from conservative after removing the shared baseline?
2. Does GSS phi-coefficient validation improve from the r~0.04 baseline?
3. Are the PC poles more stable across layers after centering?
4. Which approach (centering vs contrastive) provides cleaner demographic axes?

### Next Steps
- **Phase 6**: LLM judge filtering for cleaner vector estimates
- **Phase 7**: Multi-turn demographic drift tracking
- **Phase 8**: Activation capping along demographic axes